In [ ]:
### Forecasting for univariate time series
# https://datascience.stackexchange.com/questions/12721/time-series-prediction-using-arima-vs-lstm
# above link was a useful discussion on the state of producing prediction/confidence
#The following notebook was adapted from 
#https://machinelearningmastery.com/time-series-forecasting-long-short-term-memory-network-python/
#to apply a "walk forward" forecasting model where a point is predicted and validated 
#then the model refit with the actual point appended (as if we are predicting only the next point at every step)

import numpy as np
import pandas as pd
import random
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from matplotlib import pyplot

In [ ]:
## import temp dataset

fdir = "C:/Users/..."
file  = "time_series.xlsx"

dataraw = pd.read_excel(fdir + '/' + file, sheet_name="data", usecols=[4])
dataclean = dataraw[:-5].copy().dropna()

outliers_fraction = .01 #fraction of normal distribution that accounts for 3, 4, 5 stds is 0.003, 0.00006, 0.0000006, respectively
n_outliers = int(outliers_fraction*dataclean.describe().loc["count","Temp"])
# create list of indexes to modify
index_outliers = pd.DataFrame(random.sample(range(0,int(dataclean.describe().loc["count","Temp"]+1)), n_outliers) )

dataclean_mean = float(dataclean.describe().loc["mean","Temp"])
dataclean_std = float(dataclean.describe().loc["std","Temp"])
#print(dataclean_mean, dataclean_std)

random.seed(18)
datadirty = dataclean.copy()
datadirty_class = np.zeros(len(dataclean))
datadirty_class = pd.DataFrame(datadirty_class, columns=['class'])

for i, row in index_outliers.iterrows():
	point_temp = float(dataclean.loc[row,"Temp"])
	# if point_temp >= dataclean_mean:
	dirty_temp = point_temp + ((random.random() - 0.5)*2*dataclean_std*5)
	# if point_temp < dataclean_mean:
	# 	dirty_temp = point_temp - (random.random()*dataclean_std*4)
	datadirty.at[row,"Temp"] = dirty_temp
	datadirty_class.at[row,"class"] = 1

datadirty_class = datadirty_class.as_matrix()
dataclean = dataclean["Temp"].as_matrix()
datadirty= datadirty["Temp"].as_matrix().copy()

In [ ]:
## data is broken into training and testing sets
# train_step and dataclean_dwn_sample are included as default to provide faster computation

train_step = int(60) # sample equivalent of 4 minutes
dataclean_dwn_samp = dataclean[::train_step].copy()

two_days = int(43200/train_step) #in samples
#two_days = int(43200) #raw time scale
train_size = two_days #in hours
n_forecast_pts = len(datadirty[::train_step]) - train_size

In [ ]:
### generate a sequence class using timeseriesgenerator on datadirty

# date-time parsing function for loading the dataset
#def parser(x):
#	return datetime.strptime('190'+x, '%Y-%m')

# frame a sequence as a supervised learning problem by shifting so that t+1 is the "solution" of t
def timeseries_to_supervised(data, lag=1):
	df = pd.DataFrame(data)
	columns = [df.shift(i) for i in range(1, lag+1)]
	columns.append(df)
	df = pd.concat(columns, axis=1)
	df.fillna(0, inplace=True)
	return df

# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return pd.Series(diff)

# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]

# scale train and test data to [-1, 1]
def scale(train, test):
	# fit scaler
	scaler = MinMaxScaler(feature_range=(-1, 1))
	scaler = scaler.fit(train)
	# transform train
	train = train.reshape(train.shape[0], train.shape[1])
	train_scaled = scaler.transform(train)
	# transform test
	test = test.reshape(test.shape[0], test.shape[1])
	test_scaled = scaler.transform(test)
	return scaler, train_scaled, test_scaled

# inverse scaling for a forecasted value
def invert_scale(scaler, X, value):
	new_row = [x for x in X] + [value]
	array = numpy.array(new_row)
	array = array.reshape(1, len(array))
	inverted = scaler.inverse_transform(array)
	return inverted[0, -1]

# fit an LSTM network to training data
def fit_lstm(train, batch_size, nb_epoch, neurons):
	X, y = train[:, 0:-1], train[:, -1]
	X = X.reshape(X.shape[0], 1, X.shape[1])
	model = Sequential()
	model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
	model.add(Dense(1))
	model.compile(loss='mean_squared_error', optimizer='adam')
	for i in range(nb_epoch):
		model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
		model.reset_states()
	return model

# make a one-step forecast
def forecast_lstm(model, batch_size, X):
	X = X.reshape(1, 1, len(X))
	yhat = model.predict(X, batch_size=batch_size)
	return yhat[0,0]

In [ ]:
## transform data to be stationary and prepare for training/predicting
#raw_values = series.values
raw_values = dataclean_dwn_samp
diff_values = difference(raw_values, 1)

# transform data to be supervised learning
supervised = timeseries_to_supervised(diff_values, 1)
supervised_values = supervised.values

# split data into train and test-sets
# train, test = supervised_values[0:-12], supervised_values[-12:]
train, test = supervised_values[0:train_size], supervised_values[train_size:]

# transform the scale of the data
scaler, train_scaled, test_scaled = scale(train, test)

In [ ]:
# the model
lstm_model = fit_lstm(train_scaled, 1, 1000, 8)
# forecast the entire training dataset to build up state for forecasting
train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)
lstm_model.predict(train_reshaped, batch_size=1)

In [ ]:
# walk-forward validation on the test data
predictions = list()
for i in range(len(test_scaled)):
	# make one-step forecast
	X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
	yhat = forecast_lstm(lstm_model, 1, X)
	# invert scaling
	yhat = invert_scale(scaler, X, yhat)
	# invert differencing
	yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
	# store forecast
	predictions.append(yhat)
	expected = raw_values[len(train) + i + 1]
	print('Hour=%d, Predicted=%f, Expected=%f' % (i+1+48, yhat, expected))

In [ ]:
## report performance
rmse = sqrt(mean_squared_error(raw_values[two_days:two_days+len(predictions)], predictions))
print(rmse.describe())

## Wilson Score Interval for computing 95% CI bounds
#https://en.wikipedia.org/wiki/Binomial_proportion_confidence_interval#Wilson_score_interval

In [ ]:
# method for generating error bounds

error = rmse.copy()
#!!! CI cannot be computed with errors larger than 1,
# the below method is more appropriate for classification error, however
# a transform has been applied to reframe RMSE in terms of perecnt of maximum RMSE observed

##
scaler_error = MinMaxScaler(feature_range=(0,1))
scaler_error = scaler_error.fit(error.reshape(-1,1))
scaled_error = scaler_error.transform(error.reshape(-1,1))
# print(scaled_error)
n = len(error)
const = 1.96 # constant associated with 95% CI interval
CIu = scaled_error + const*np.sqrt((scaled_error * (1 - scaled_error)) / n)
CIl = scaled_error - const*np.sqrt((scaled_error * (1 - scaled_error)) / n)
##
#!!! the following may work to convert errors back into RMSE scale, however
# I don't believe this is a proper inversion for the desired outcome
CIu_rmse = scaler_error.inverse_transform(CIu)
CIl_rmse = scaler_error.inverse_transform(CIl)

print('Test RMSE: %.3f' % rmse)

In [ ]:
# line plot of observed vs predicted
pyplot.plot(raw_values[two_days:])
pyplot.plot(predictions)
pyplot.plot(predictions+CIu_rmse, linestyle='--')
pyplot.plot(predictions-CIl_rmse, linestyle='--')
pyplot.show()